In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

# Initialize Spark session
spark = SparkSession.builder.appName("LipstickPrediction").getOrCreate()

# MySQL database connection settings
mysql_host = "localhost"
mysql_port = "3306"
mysql_database = "tintandtone"
mysql_username = "your_username"
mysql_password = "your_password"

# MySQL JDBC URL
jdbc_url = f"jdbc:mysql://{mysql_host}:{mysql_port}/{mysql_database}"

# Table and column names
table_name = "skin"
column_rgb = "rgb"
column_skintone = "skintone"
column_undertone = "undertone"
column_lipstick = "lipstick"

# Load data from MySQL table
data = spark.read.format("jdbc").options(
    url=jdbc_url,
    driver="com.mysql.jdbc.Driver",
    dbtable=table_name,
    user=mysql_username,
    password=mysql_password
).load()

# Perform necessary data preprocessing steps such as cleaning, transforming, etc.
# You can use Spark SQL or DataFrame operations to manipulate the data if needed.

# Define the feature columns and target column
feature_columns = [column_rgb, column_skintone, column_undertone]
target_column = column_lipstick

# Create a vector assembler to combine the feature columns into a feature vector
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data)

# Initialize the random forest classifier
rf = RandomForestClassifier(labelCol=target_column, featuresCol="features")

# Train the model
model = rf.fit(data)

# Function to predict lipstick based on skin tone
def predict_lipstick(skin_tone_rgb, skin_tone, undertone):
    # Create a DataFrame with a single row containing the input data
    input_data = spark.createDataFrame([(skin_tone_rgb, skin_tone, undertone)], [column_rgb, column_skintone, column_undertone])
    input_data = assembler.transform(input_data)

    # Make predictions
    predictions = model.transform(input_data)

    # Extract the predicted lipstick
    predicted_lipstick = predictions.select(target_column).first()[0]
    return predicted_lipstick

# Example usage
skin_tone_rgb = "FFFFFF"  # RGB value of the skin tone
skin_tone = "Fair"  # Skin tone category
undertone = "Cool"  # Undertone category

predicted_lipstick = predict_lipstick(skin_tone_rgb, skin_tone, undertone)
print("Predicted Lipstick: ", predicted_lipstick)

# Stop the Spark session
spark.stop()
